In [1]:
import pandas as pd 
import numpy as np
import df_helpers as helper
from sanbomics.tools import id_map


import os
dirs = helper.dirs()

### Construct gene info tables + significant DEGs

In [3]:
DCTRL0_upsigs_df = helper.read_sigs('Up','DCTRL0')
DCTRL0_downsigs_df = helper.read_sigs('Down','DCTRL0')

batchA_df = pd.read_excel(f'{dirs.batchcounts}/A_gene_counts.xlsx')
batchA_df.rename(columns={'gene_name':'Symbol'},inplace=True)

## Create df with relevant locations of gene and promoter + on what chromosome and strand
genedata_df = batchA_df[['Symbol','gene_start','gene_end','gene_chr','gene_strand']]
genedata_df['Promoter_start'] = genedata_df.gene_start - 1000
genedata_df['Promoter_end'] = genedata_df.gene_start + 100

# subset genedata_df base on significant genes
res_up = DCTRL0_upsigs_df.merge(genedata_df,how='inner',on='Symbol')
res_down = DCTRL0_downsigs_df.merge(genedata_df,how='inner',on='Symbol')

FileNotFoundError: [Errno 2] No such file or directory: '1. Batch Counts/A_gene_counts.xlsx'

In [15]:
res

,Symbol,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,gene_start,gene_end,gene_chr,gene_strand,Promoter_start,Promoter_end
0,DDIT4,2312.933120,3.626941,0.310929,11.664842,1.927613e-31,3.365806e-27,72273920,72276036,10,+,72272920,72274020
1,PFKFB4,1156.457297,2.210382,0.205462,10.758115,5.426893e-27,2.368975e-23,48517684,48562015,3,-,48516684,48517784
2,BHLHE40,656.289464,2.391624,0.221910,10.777457,4.398797e-27,2.368975e-23,4979116,4985323,3,+,4978116,4979216
3,PPFIA4,945.833115,2.614851,0.250851,10.423915,1.928480e-25,6.734637e-22,203026498,203078740,1,+,203025498,203026598
4,MIR210HG,190.890887,3.874204,0.384548,10.074706,7.147440e-24,2.080024e-20,565660,568457,11,-,564660,565760
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1084,AC011472.1,35.521404,1.104751,0.440579,2.507499,1.215890e-02,4.937363e-02,11203628,11216168,19,+,11202628,11203728
1085,CALCRL,30.278869,1.271135,0.507356,2.505410,1.223097e-02,4.951656e-02,187343129,187448460,2,-,187342129,187343229
1086,SRP54-AS1,26.816752,1.135093,0.453043,2.505489,1.222823e-02,4.951656e-02,34920858,34982532,14,-,34919858,34920958
1087,TSHZ2,42.171128,1.166966,0.465843,2.505065,1.224289e-02,4.951889e-02,52972407,53495330,20,+,52971407,52972507


### Fetch Promoter Sequences

In [16]:
from Bio import Entrez, SeqIO
## will take 80 min to go through entire upregulated df
#create NCBI Chromosome accession lookup table
chromosome_accession_dict = {
    '1': 'NC_000001.11',
    '2': 'NC_000002.12',
    '3': 'NC_000003.12',
    '4': 'NC_000004.12',
    '5': 'NC_000005.10',
    '6': 'NC_000006.12',
    '7': 'NC_000007.14',
    '8': 'NC_000008.11',
    '9': 'NC_000009.12',
    '10': 'NC_000010.11',
    '11': 'NC_000011.10',
    '12': 'NC_000012.12',
    '13': 'NC_000013.11',
    '14': 'NC_000014.9',
    '15': 'NC_000015.10',
    '16': 'NC_000016.10',
    '17': 'NC_000017.11',
    '18': 'NC_000018.10',
    '19': 'NC_000019.10',
    '20': 'NC_000020.11',
    '21': 'NC_000021.9',
    '22': 'NC_000022.11',
    'X': 'NC_000023.11',
    'Y': 'NC_000024.10',
}

# This function calls entrez fetch function from biopython to fetch sequence that matches region of interest specified
def extract_dnasequence(gene_chr,gene_strand,Promoter_start,Promoter_end):

    # gene_chr = 3; Promoter_start = 4978116; Promoter_end = 4979216 ; gene_strand = '+'
    genestrand_conversion = lambda x: '1' if x =='+' else '2' if x =='+' else 0
    # Always provide your email when accessing NCBI services
    Entrez.email = "ae15@stanford.edu"

    # Define the parameters for the sequence you want to fetch
    db = "nucleotide"  # Database
    # Use Entrez.efetch to get the sequence
    handle = Entrez.efetch(db='nucleotide', 
                           id=chromosome_accession_dict[str(gene_chr)], 
                           rettype="fasta", 
                           strand= genestrand_conversion(gene_strand),
                           seq_start=Promoter_start, 
                           seq_stop=Promoter_end)

    # Parse the fetched sequence
    sequence = SeqIO.read(handle, "fasta")
    handle.close()

    # Print the fetched sequence
    # print(sequence.id)
    # print(sequence.seq)
    return sequence.seq

extraction_cols = ['gene_chr','gene_strand','Promoter_start','Promoter_end']
promoters = []
for i in res.index:
    input = res[extraction_cols].loc[i].to_list()
    if str(input[0]) in chromosome_accession_dict.keys():
        promoters.append(extract_dnasequence(*input))


In [8]:
res[extraction_cols].loc[i].to_list()

[18, '-', 653785, 654035]

In [17]:

# save promoters
gene_symbols = res.Symbol.to_list()
df = dict(zip(gene_symbols,[str(p) for p in promoters]))
promoter_df = pd.DataFrame.from_dict(data=df,orient='index')
promoter_df.to_excel(f'{dirs.tfs}/promoters.xlsx')


### Export as FASTA

In [18]:
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO
# Define your sequences

gene_symbols = res.Symbol.to_list()

promoters_df = pd.read_excel(f'{dirs.tfs}/promoters.xlsx')
gene_symbols  = promoters_df[promoters_df.columns[0]].to_list()
promoters = promoters_df[promoters_df.columns[1]].to_list()

fasta_list = []
for i in range(len(promoters)):
    fasta_list.append(SeqRecord(Seq(promoters[i]), id=f"seq{i}", description=f"{gene_symbols[i]} promoter region (TSS-1000 to TSS+100)"))


SeqIO.write(fasta_list, f'{dirs.tfs}/promoters.fasta', "fasta")

1085

## HOMER

## Save significant gene symbols as text

In [11]:
res = helper.read_results('Up','DCTRL0')
res.reset_index(inplace=True)
res

,Symbol,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
0,PCNX2,937.604003,3.518934,0.103609,33.963677,7.663003e-253,2.451012e-248
1,MFGE8,1089.284818,6.472362,0.199294,32.476525,2.287704e-231,3.658611e-227
2,THY1,2482.720685,10.290578,0.320214,32.136565,1.360802e-226,1.450842e-222
3,DPYSL2,3204.528640,3.917715,0.136434,28.715141,2.468981e-181,1.128148e-177
4,FGFR1,3029.223719,3.808899,0.132898,28.660360,1.190872e-180,4.761257e-177
...,...,...,...,...,...,...,...
8165,SPTBN5,40.584868,0.000891,0.223130,0.003991,9.968156e-01,1.000000e+00
8166,GP6,73.259858,0.000957,0.450079,0.002126,9.983035e-01,1.000000e+00
8167,NPM1P9,12.613348,0.001094,0.428777,0.002551,9.979644e-01,1.000000e+00
8168,EBF2,12.278734,0.001097,0.613201,0.001789,9.985727e-01,1.000000e+00


In [18]:
symbols_ls = res.Symbol.to_list()
with open('my_list.txt', 'w') as file:
    for item in symbols_ls:
        file.write(str(item) + '\n')


In [204]:
len(symbols_ls)

1089

In [26]:
from Bio import Entrez
from Bio import SeqIO

def get_genomic_info(gene_symbol):
    Entrez.email = 'ae115@stanford.edu'  # Specify your email for Entrez

    # Search for the gene symbol in the Entrez Gene database
    handle = Entrez.esearch(db='gene', term=f'{gene_symbol}')
    record = Entrez.read(handle)
    gene_id = record['IdList'][0]

    # Retrieve genomic information for the gene from the Ensembl database
    handle = Entrez.efetch(db='gene', id=gene_id, rettype='gb', retmode='text')
    gene_record = SeqIO.read(handle, 'genbank')

    return gene_record

# Example usage:
gene_symbol = 'BRCA1'  # Example gene symbol
gene_record = get_genomic_info(gene_symbol)
if gene_record:
    print(gene_record)
else:
    print("Failed to retrieve genomic information for the gene.")


ValueError: No records found in handle